In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
995371,2021-02-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
995372,2021-02-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
995373,2021-02-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
995374,2021-02-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27091,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27093,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27095,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27097,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27099,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
617429,2021-02-13,Arkansas,Arkansas,5001,1919,37.00,5000,Arkansas,AR,Arkansas,State,3017804
617431,2021-02-14,Arkansas,Arkansas,5001,1926,37.00,5000,Arkansas,AR,Arkansas,State,3017804
617433,2021-02-15,Arkansas,Arkansas,5001,1929,37.00,5000,Arkansas,AR,Arkansas,State,3017804
617435,2021-02-16,Arkansas,Arkansas,5001,1930,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
995371,2021-02-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
995372,2021-02-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
995373,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
995374,2021-02-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-17') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
393,2021-02-17,Snohomish,Washington,30130,520.00,53061,WA,County,822083,3665.08,63.25
784,2021-02-17,Cook,Illinois,467424,9747.00,17031,IL,County,5150233,9075.78,189.25
1174,2021-02-17,Orange,California,257816,3644.00,6059,CA,County,3175692,8118.42,114.75
1563,2021-02-17,Maricopa,Arizona,500907,8580.00,4013,AZ,County,4485414,11167.46,191.29
1952,2021-02-17,Los Angeles,California,1172221,19368.00,6037,CA,County,10039107,11676.55,192.93
...,...,...,...,...,...,...,...,...,...,...,...
994987,2021-02-17,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
995115,2021-02-17,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
995212,2021-02-17,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
995305,2021-02-17,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
393,2021-02-17,Snohomish,Washington,30130,520.00,53061,WA,County,822083,3665.08,63.25,63.25,3665.08
784,2021-02-17,Cook,Illinois,467424,9747.00,17031,IL,County,5150233,9075.78,189.25,189.25,9075.78
1174,2021-02-17,Orange,California,257816,3644.00,6059,CA,County,3175692,8118.42,114.75,114.75,8118.42
1563,2021-02-17,Maricopa,Arizona,500907,8580.00,4013,AZ,County,4485414,11167.46,191.29,191.29,11167.46
1952,2021-02-17,Los Angeles,California,1172221,19368.00,6037,CA,County,10039107,11676.55,192.93,192.93,11676.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994987,2021-02-17,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
995115,2021-02-17,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
995212,2021-02-17,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
995305,2021-02-17,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
702265,2021-02-17,Clarke,Alabama,3413,47.00,1025,AL,County,23622,14448.40,198.97,198.97,14448.40,1
645470,2021-02-17,Hale,Alabama,2031,62.00,1065,AL,County,14651,13862.53,423.18,423.18,13862.53,2
505581,2021-02-17,Lowndes,Alabama,1296,44.00,1085,AL,County,9726,13325.11,452.40,452.40,13325.11,3
448497,2021-02-17,Etowah,Alabama,12995,300.00,1055,AL,County,102268,12706.81,293.35,293.35,12706.81,4
376596,2021-02-17,Franklin,Alabama,3949,75.00,1059,AL,County,31362,12591.67,239.14,239.14,12591.67,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616459,2021-02-17,Sublette,Wyoming,669,7.00,56035,WY,County,9831,6805.00,71.20,71.20,6805.00,19
947076,2021-02-17,Platte,Wyoming,568,11.00,56031,WY,County,8393,6767.54,131.06,131.06,6767.54,20
850733,2021-02-17,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
765057,2021-02-17,Lincoln,Wyoming,1184,11.00,56023,WY,County,19830,5970.75,55.47,55.47,5970.75,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
505581,2021-02-17,Lowndes,Alabama,1296,44.00,1085,AL,County,9726,13325.11,452.40,452.40,13325.11,3,1
645470,2021-02-17,Hale,Alabama,2031,62.00,1065,AL,County,14651,13862.53,423.18,423.18,13862.53,2,2
447177,2021-02-17,Clay,Alabama,1403,53.00,1027,AL,County,13235,10600.68,400.45,400.45,10600.68,22,3
545058,2021-02-17,Greene,Alabama,857,32.00,1063,AL,County,8111,10565.90,394.53,394.53,10565.90,24,4
215662,2021-02-17,Walker,Alabama,6384,236.00,1127,AL,County,63521,10050.22,371.53,371.53,10050.22,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543745,2021-02-17,Hot Springs,Wyoming,358,3.00,56017,WY,County,4413,8112.40,67.98,67.98,8112.40,16,19
746118,2021-02-17,Uinta,Wyoming,2016,12.00,56041,WY,County,20226,9967.37,59.33,59.33,9967.37,5,20
765057,2021-02-17,Lincoln,Wyoming,1184,11.00,56023,WY,County,19830,5970.75,55.47,55.47,5970.75,22,21
214654,2021-02-17,Teton,Wyoming,3298,9.00,56039,WY,County,23464,14055.57,38.36,38.36,14055.57,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.25,3665.08,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.25,3665.08,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.25,3665.08,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.25,3665.08,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.25,3665.08,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993019,2021-02-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
993020,2021-02-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
993021,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
993022,2021-02-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
699591,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,198.97,14448.40,45,1,2.00
699592,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,198.97,14448.40,45,1,0.00
699593,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,198.97,14448.40,45,1,1.00
699594,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,198.97,14448.40,45,1,4.00
699595,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,198.97,14448.40,45,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777741,2021-02-13,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
777742,2021-02-14,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
777743,2021-02-15,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
777744,2021-02-16,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
503555,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13325.11,1,3,1.00,0.00
503556,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13325.11,1,3,0.00,0.00
503557,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13325.11,1,3,0.00,0.00
503558,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13325.11,1,3,0.00,0.00
503559,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13325.11,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501905,2021-02-13,Albany,Wyoming,3849,11.00,56001,WY,County,38880,9899.69,28.29,28.29,9956.28,23,6,0.00,0.00
501906,2021-02-14,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9956.28,23,6,5.00,0.00
501907,2021-02-15,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9956.28,23,6,0.00,0.00
501908,2021-02-16,Albany,Wyoming,3867,11.00,56001,WY,County,38880,9945.99,28.29,28.29,9956.28,23,6,13.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-17') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
257294,2021-02-17,Imperial,California,26797,583.00,6025,CA,County,181215,14787.41,321.72,321.72,14787.41,1,2,18.00,0.00
1952,2021-02-17,Los Angeles,California,1172221,19368.00,6037,CA,County,10039107,11676.55,192.93,192.93,11676.55,2,5,2121.00,153.00
449769,2021-02-17,Inyo,California,1191,34.00,6027,CA,County,18039,6602.36,188.48,188.48,6602.36,3,31,7.00,0.00
57553,2021-02-17,Stanislaus,California,54631,910.00,6099,CA,County,550660,9921.00,165.26,165.26,9921.00,4,10,147.00,5.00
71310,2021-02-17,Tulare,California,46444,697.00,6107,CA,County,466195,9962.35,149.51,149.51,9962.35,5,9,100.00,6.00
26742,2021-02-17,Riverside,California,287063,3577.00,6065,CA,County,2470546,11619.42,144.79,144.79,11619.42,6,6,473.00,19.00
545000,2021-02-17,Kings,California,21696,213.00,6031,CA,County,152940,14185.96,139.27,139.27,14185.96,7,3,38.00,4.00
407749,2021-02-17,Merced,California,28306,382.00,6047,CA,County,277680,10193.75,137.57,137.57,10193.75,8,8,105.00,0.00
26046,2021-02-17,Fresno,California,93363,1341.00,6019,CA,County,999101,9344.70,134.22,134.22,9344.70,9,14,132.00,50.00
45825,2021-02-17,San Joaquin,California,65659,1018.00,6077,CA,County,762148,8614.99,133.57,133.57,8614.99,10,18,300.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
946627,2021-02-17,Lassen,California,5546,19.00,6035,CA,County,30573,18140.19,62.15,62.15,18140.19,36,1,0.00,0.00
257294,2021-02-17,Imperial,California,26797,583.00,6025,CA,County,181215,14787.41,321.72,321.72,14787.41,1,2,18.00,0.00
545000,2021-02-17,Kings,California,21696,213.00,6031,CA,County,152940,14185.96,139.27,139.27,14185.96,7,3,38.00,4.00
128400,2021-02-17,San Bernardino,California,282736,2457.00,6071,CA,County,2180085,12969.04,112.70,112.70,12969.04,13,4,242.00,39.00
1952,2021-02-17,Los Angeles,California,1172221,19368.00,6037,CA,County,10039107,11676.55,192.93,192.93,11676.55,2,5,2121.00,153.00
26742,2021-02-17,Riverside,California,287063,3577.00,6065,CA,County,2470546,11619.42,144.79,144.79,11619.42,6,6,473.00,19.00
161325,2021-02-17,Kern,California,100849,690.00,6029,CA,County,900202,11202.93,76.65,76.65,11202.93,27,7,263.00,3.00
407749,2021-02-17,Merced,California,28306,382.00,6047,CA,County,277680,10193.75,137.57,137.57,10193.75,8,8,105.00,0.00
71310,2021-02-17,Tulare,California,46444,697.00,6107,CA,County,466195,9962.35,149.51,149.51,9962.35,5,9,100.00,6.00
57553,2021-02-17,Stanislaus,California,54631,910.00,6099,CA,County,550660,9921.00,165.26,165.26,9921.00,4,10,147.00,5.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
257294,2021-02-17,Imperial,California,26797,583.00,6025,CA,County,181215,14787.41,321.72,321.72,14787.41,1,2,18.00,0.00
1952,2021-02-17,Los Angeles,California,1172221,19368.00,6037,CA,County,10039107,11676.55,192.93,192.93,11676.55,2,5,2121.00,153.00
449769,2021-02-17,Inyo,California,1191,34.00,6027,CA,County,18039,6602.36,188.48,188.48,6602.36,3,31,7.00,0.00
57553,2021-02-17,Stanislaus,California,54631,910.00,6099,CA,County,550660,9921.00,165.26,165.26,9921.00,4,10,147.00,5.00
71310,2021-02-17,Tulare,California,46444,697.00,6107,CA,County,466195,9962.35,149.51,149.51,9962.35,5,9,100.00,6.00
26742,2021-02-17,Riverside,California,287063,3577.00,6065,CA,County,2470546,11619.42,144.79,144.79,11619.42,6,6,473.00,19.00
545000,2021-02-17,Kings,California,21696,213.00,6031,CA,County,152940,14185.96,139.27,139.27,14185.96,7,3,38.00,4.00
407749,2021-02-17,Merced,California,28306,382.00,6047,CA,County,277680,10193.75,137.57,137.57,10193.75,8,8,105.00,0.00
26046,2021-02-17,Fresno,California,93363,1341.00,6019,CA,County,999101,9344.70,134.22,134.22,9344.70,9,14,132.00,50.00
45825,2021-02-17,San Joaquin,California,65659,1018.00,6077,CA,County,762148,8614.99,133.57,133.57,8614.99,10,18,300.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14787.41,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3362,02/13/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00
3363,02/14/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00
3364,02/15/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00
3365,02/16/21,Lassen,5546,19.00,18140.19,62.15,62.15,18140.19,36,1,48.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14787.41,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14787.41,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3436,02/13/21,San Joaquin,64969,992.00,8524.46,130.16,133.57,8614.99,10,18,0.00,0.00
3437,02/14/21,San Joaquin,64969,992.00,8524.46,130.16,133.57,8614.99,10,18,0.00,0.00
3438,02/15/21,San Joaquin,64969,992.00,8524.46,130.16,133.57,8614.99,10,18,0.00,0.00
3439,02/16/21,San Joaquin,65359,1018.00,8575.63,133.57,133.57,8614.99,10,18,390.00,26.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)